In [1]:
import pandas as pd
import re
import json
from collections import OrderedDict 
import sys

sys.path.insert(1, 'C:\\Users\\dharmapu\\Documents\\personal\\ui\\KA-AMSD_src\\src\\text_processing\\preprocess')
import preprocess as pre

In [2]:
%%time
f_teks_ori = 'C:\\Users\\dharmapu\\Documents\\personal\\ui\\KA-AMSD_src\\paper-submission\\data_untuk_anotasi\\gempa\\v01\\sample_gempa.xlsx'
df_teks_ori = pd.read_excel(f_teks_ori)
df_analysis = df_teks_ori[['id', 'source', 'title']].copy()

Wall time: 394 ms


In [3]:
 def pre_news_raw_text(news_id):
    title_text = df_analysis.loc[df_analysis['id'] == news_id, 'title'].values[0]
    src_text = df_analysis.loc[df_analysis['id'] == news_id, 'source'].values[0]
    src_text = src_text + ".json"
    text_dir = 'C:\\Users\\dharmapu\\documents\\personal\\ui\\KA-AMSD_src\\final_data\\hasil-scraping\\complete'
    text_path = text_dir + "\\" + src_text

    special = r"(\W|\\u[\w][\w][\w][\w])*"
   
    title_text = title_text.replace('[', '\[')
    title_text = title_text.replace(']', '\]')
    title_text = title_text.replace('?', '\?')
    title_text = title_text.replace('(', '\(')
    title_text = title_text.replace(')', '\)')
    title_text = title_text.replace('.', '\.')
    title_text = title_text.replace(',', '\,')
    title_text = title_text.replace('"', '\",')
    
    title_text = title_text.replace(' ', special)
    re_title = r"title\":.*" + title_text + r".*(date|content)\""
    
    l_text = []
    with open(text_path) as f:
        l_text = [l for l in f if re.search(re_title, l)]

        for index, text in enumerate(l_text):
            tmp_text = text
            tmp_text = pre.clean_convertion(tmp_text)
            tmp_text = pre.clean_links_text(tmp_text)
            tmp_text = pre.clean_text(tmp_text)
            tmp_text = pre.clean_escape_char(tmp_text)

            ''' Clean up return line '''
            escape_char = re.compile(r'\\n')
            tmp_text = re.sub(escape_char, ' ', tmp_text, re.MULTILINE|re.IGNORECASE).strip()

            escape_char = re.compile(r'\\n')
            tmp_text = re.sub(escape_char, ' ', tmp_text, re.MULTILINE|re.IGNORECASE).strip()

            escape_char = re.compile(r'\\n')
            tmp_text = re.sub(escape_char, ' ', tmp_text, re.MULTILINE|re.IGNORECASE).strip()

            l_text[index] = tmp_text
                
    return list(OrderedDict.fromkeys(l_text))

In [4]:
%%time
def prep_news(l_text):
    l_news = []
    for t in l_text:
        text = t
        text = re.match('{.*}', text)
        
        try:
            json_obj = json.loads(text[0])
            content = json_obj['content']
            l_news.append(content)
        except Exception as e:
            if hasattr(e, 'message'):
                print(e.message)
            else:
                print(e)

    return ' // '.join(l_news)

Wall time: 0 ns


In [5]:
df_result = pd.DataFrame(columns = ['id', 'source', 'date', 'title', 'text'])
df_result['source'] = df_teks_ori['source'].copy()
df_result['date'] = df_teks_ori['date'].copy()

In [6]:
%%time
for index, row in df_analysis.iterrows():
    df_result.loc[index, 'id'] = df_analysis.loc[index, 'id']    
    df_result.loc[index, 'title'] = df_analysis.loc[index, 'title']
    
    text_id = df_analysis.loc[index, 'id']
    print(text_id)
    l_text = pre_news_raw_text(text_id)
    text = prep_news(l_text)
    df_result.loc[index, 'text'] = text

gempa_0001
gempa_0002
gempa_0003
gempa_0004
gempa_0005
gempa_0006
gempa_0007
gempa_0008
gempa_0009
gempa_0010
gempa_0011
gempa_0012
gempa_0013
gempa_0014
gempa_0015
gempa_0016
gempa_0017
gempa_0018
gempa_0019
gempa_0020
gempa_0021
gempa_0022
gempa_0023
gempa_0024
gempa_0025
gempa_0026
gempa_0027
gempa_0028
gempa_0029
gempa_0030
gempa_0031
gempa_0032
gempa_0033
gempa_0034
gempa_0035
gempa_0036
gempa_0037
gempa_0038
gempa_0039
gempa_0040
gempa_0041
gempa_0042
gempa_0043
gempa_0044
gempa_0045
gempa_0046
gempa_0047
gempa_0048
gempa_0049
gempa_0050
gempa_0051
gempa_0052
gempa_0053
gempa_0054
gempa_0055
gempa_0056
gempa_0057
gempa_0058
gempa_0059
gempa_0060
gempa_0061
gempa_0062
gempa_0063
gempa_0064
gempa_0065
gempa_0066
gempa_0067
gempa_0068
gempa_0069
gempa_0070
gempa_0071
gempa_0072
gempa_0073
gempa_0074
gempa_0075
gempa_0076
gempa_0077
gempa_0078
gempa_0079
gempa_0080
gempa_0081
gempa_0082
gempa_0083
gempa_0084
gempa_0085
gempa_0086
gempa_0087
gempa_0088
gempa_0089
gempa_0090
gempa_0091

In [7]:
df_result

,id,source,date,title,text
0,gempa_0001,aceh_harianrakyat,"January 9, 2020",Kerugian Gempa Simeulue Rp130 Juta,SIMEULUE (RA) Dampak bencana gempa bumi Sime...
1,gempa_0002,aceh_harianrakyat,"December 20, 2016",Duka Korban Gempa Pidie Jaya,"Dulu Dia yang mencari nafkah untuk keluarga, ..."
2,gempa_0003,aceh_harianrakyat,"October 8, 2018","Gempa 5,9 SR Melanda Haiti, 14 Orang Tewas",Harianrakyataceh. com Setidaknya 14 orang te...
3,gempa_0004,aceh_harianrakyat,"December 7, 2016","Pidie Jaya Gempa, Terasa Hingga Banda Aceh","BANDA ACEH (RA) Sekitar pukul 05.03 WIB, gemp..."
4,gempa_0005,aceh_harianrakyat,"September 30, 2018","Pascagempa Palu dan Donggala, 3 Daerah di Sult...",Harianrakyataceh. com Daerah terdampak gempa...
...,...,...,...,...,...
1212,gempa_1213,surabaya_tribun,"Senin, 2 Oktober 2017 19:39",Bim Sala Bim! Lihatlah Perubahan Pria yang Ope...,"SURYA. CO. ID - Istilah ""Ganteng itu relatif, ..."
1213,gempa_1214,surabaya_tribun,"Selasa, 19 November 2019 18:53",Sinopsis Film San Andreas Tayang di Trans TV R...,SURYA. CO. ID - Simak sinopsis film San Andrea...
1214,gempa_1215,surabaya_tribun,"Rabu, 3 Oktober 2018 18:07",Pemkot Surabaya Packing Bantuan Untuk Dikirim ...,SURYA. co. id | SURABAYA - Setelah tiga hari m...
1215,gempa_1216,surabaya_tribun,"Kamis, 26 September 2019 17:43",Siswa SD di Blitar yang Jatuh ke Sumur Diduga ...,"SURYA. co. id | BLITAR - Anisa Shela, siswi ke..."


In [8]:
df_result.to_excel('result_gempa.xlsx', index=False)